In [91]:
import pandas as pd
import tensorflow as tf
import numpy as np
from  sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [92]:
titanicTrain = pd.read_csv('D:/GitProjects/MachineLearning/TitanicDisaster/input/train.csv')
titanicTest = pd.read_csv('D:/GitProjects/MachineLearning/TitanicDisaster/input/test.csv')

In [93]:
titanicTrain.drop(['Cabin'],axis=1, inplace = True)
titanicTest.drop(['Cabin'],axis=1, inplace = True)

In [94]:
titanicTrain['Age'].fillna(titanicTrain['Age'].median(), inplace =True)
titanicTest['Age'].fillna(titanicTest['Age'].median(), inplace =True)
titanicTest['Fare'].fillna(titanicTest['Fare'].median(), inplace = True)

In [95]:
titanicTrain.drop(['Name'],axis=1, inplace=True)
titanicTest.drop(['Name'],axis=1, inplace=True)

In [96]:
mapping = {
    'female':1,
    'male': 0
}
titanicTrain['Sex'] = titanicTrain['Sex'].map(mapping)
titanicTest['Sex'] = titanicTest['Sex'].map(mapping)

In [97]:
X = titanicTrain[['Pclass', 'Age', 'SibSp', 'Sex', 'Parch', 'Fare']].as_matrix()
Y = titanicTrain['Survived'].as_matrix()
Y = Y.reshape([Y.shape[0], 1]) # Reshaping from (891,) to (891,1)
print (X.shape)
print (Y.shape)
x = titanicTest[['Pclass', 'Age', 'SibSp', 'Sex', 'Parch', 'Fare']].as_matrix()
#y = titanicTest['Survived'].to_matrix()
#y = y.reshape([y.shape[0], 1]) # Reshaping from (891,) to (891,1)
print(x.shape)


(891, 6)
(891, 1)
(418, 6)


In [98]:
poly = PolynomialFeatures(2)
X = poly.fit_transform(X)
x = poly.fit_transform(x)
print(X.shape)
print(x.shape)

(891, 28)
(418, 28)


In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, Y )
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(668, 28)
(223, 28)
(668, 1)


In [100]:
# Model

lr = 0.01
batch_size = 128
n_epochs = 2500

X_hold = tf.placeholder(tf.float32,[None,X.shape[1]])
Y_hold = tf.placeholder(tf.float32,[None,Y.shape[1]])



w = tf.Variable(tf.truncated_normal(shape=[X.shape[1], Y.shape[1]], stddev=0.01), name = "w")
b = tf.Variable(tf.zeros([1,1]),'b')

logits = tf.matmul(X_hold,w)+b
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels = Y_hold))
opt = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)

cp = tf.equal(tf.argmax(logits, axis=1), tf.cast(Y_hold, dtype=tf.int64))
acc = tf.reduce_mean(tf.cast(cp,tf.float32))
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)


In [1]:
for i in tqdm(range(10000)):
    #if (len(y_train.shape)) == 1:
    #    y_train = y_train.reshape((y_train.shape[0],1))
    sess.run(opt,feed_dict={X_hold: X_train, Y_hold: y_train})
   # if i %100 == 0:
    #    if len(y_train.shape) == 1:
    #        y_train = y_train.reshape((y_train[0],1))
        #l = loss.eval(feed_dict={X_hold:X_train, Y_hold: y_train})
a = acc.eval(feed_dict={X_hold: X_test, Y_hold: y_test})

print (a)

NameError: name 'tqdm' is not defined

In [109]:
sigm = tf.nn.sigmoid(logits, name='pred')

pred = tf.cast(tf.greater_equal(sigm, 0.8), tf.float32, name='pred') # 1 if >= 0.5

In [110]:
predictions = sess.run(pred, feed_dict={X_hold:x}).astype(int)
#print(x.shape)
#predictions

In [111]:
test_id = titanicTest['PassengerId'].values
test_id = test_id.reshape(-1,1)
#test_id.head()

In [112]:
prediction = pd.DataFrame(np.concatenate([test_id, predictions], axis=1),
                          columns=['PassengerId', 'Survived'])

In [113]:
prediction


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [114]:
import csv


In [115]:
prediction.to_csv("predictions.csv", index=False)